In [1]:
import instructor
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from pydantic import BaseModel, Field
import pandas as pd
from IPython.display import display, Markdown

In [2]:
vertexai.init(location="europe-west4")

client = instructor.from_vertexai(
    client=GenerativeModel("gemini-2.0-flash"),
    mode=instructor.Mode.VERTEXAI_JSON,
)

In [ ]:
from pathlib import Path

pdf_path = Path("../data/ad_annual-report_2024_interactive.pdf")
data_bytes = pdf_path.read_bytes()

report = Part.from_data(
    data=data_bytes,
    mime_type="application/pdf",
)

In [4]:
class AnswerSource(BaseModel):
    page: int = Field(description="The page number in the source document where the answer is found.")
    citation: str = Field(description="An exact text citation of the answer in the source document.")
    

class AuditAnswer(BaseModel):
    question: str
    answer: str = Field(..., description="The extracted answer to the question.")
    sources: list[AnswerSource] = Field(description="Pieces of evidence from the source document that support the answer.")

class AuditAnswers(BaseModel):
    answers: list[AuditAnswer]

prompt: str = """
Answer the following questions:
Could you specify the time frame these financial statements represent? 
What accounting standards or principles were applied in the preparation of these statements? 
Have there been any alterations to accounting policies or practices since prior reporting periods? 
What process is used to determine and reveal major accounting estimates? 
What critical assumptions and judgments were utilized in the preparation of these financial statements? 
How would you describe the organization's financial standing and performance as reflected in these statements? 
Do these financial results include any extraordinary or one-time items? 
Are there any transactions with related parties that are documented in the financial statements? 
Could you detail the principal elements of operating expenses and their classification? 
Are there any asset impairments or write-downs shown in these statements? 
What accounting treatment is applied to changes in fair value, particularly for financial instruments? 
Do the financial statements disclose any contingent liabilities or ongoing legal issues? 
Are there any post-reporting period events that might affect the financial results? 
How do these statements depict the organization's liquidity and cash flow situation? 
Have there been any notable shifts in working capital ratios relative to previous periods? 
Could you describe the organization's debt composition and the presentation of interest expenses? 
What accounting and presentation methods are used for taxes in these financial statements? 
Are there any contractual commitments or obligations outlined in these statements? 
Could you provide a detailed breakdown of equity, including movements in retained earnings? 
If relevant, what accounting treatment is applied to foreign currency transactions and translation? 
Are there any significant differences or contradictions found within these financial statements? 
Could you explain the organization's approach to recognizing revenue and its implementation? 
What process is used to determine and disclose provisions for doubtful accounts and bad debts? 
Are there any major events or transactions that might affect the ability to compare these statements with others? 
How does the organization handle risk management, and how is this communicated? 
What accounting treatment is used for gains or losses from asset disposals in these statements? 
Are there any non-operating revenues or expenses that influence these financial statements? 
Could you describe the nature and extent of the audit or review procedures carried out on these statements? 
... using the attached annual statement PDF.
"""

audit_answers: AuditAnswers = client.create(
    messages=[
        prompt,
        report
    ],
    response_model=AuditAnswers,
)
audit_answers

AuditAnswers(answers=[AuditAnswer(question='Could you specify the time frame these financial statements represent?', answer='The financial statements represent the financial year ending on December 29, 2024, which consisted of 52 weeks.', sources=[AnswerSource(page=42, citation='Net sales for the financial year ending on December 29, 2024, were €89.4bn, an increase of 0.7% compared to net sales for the year ending on December 31, 2023.'), AnswerSource(page=42, citation='Net sales for the financial year ending on December 29, 2024, were €89,356 million, an increase of €622 million, or 0.7%, compared to net sales of €88,734 million for the financial year ending on December 31, 2023.'), AnswerSource(page=86, citation="Ahold Delhaize's financial year is a 52- or 53-week period ending on the Sunday nearest to December 31 for the Company and our European operations, or the Saturday before the Sunday nearest to December 31 for our operations in the United States. The financial year 2024 consi

In [6]:
text_markdown = ""
text_markdown += "# Audit Q/A Report\n"
text_markdown += "> Ahold Delhaize Annual Report 2024\n"
for answer in audit_answers.answers:
    text_markdown += f"## {answer.question}\n"
    text_markdown += f"{answer.answer}\n"
    for source in answer.sources:
        text_markdown += f"> **Page {source.page}**\n"
        text_markdown += f"> {source.citation}\n"
        text_markdown += "\n"

display(Markdown(text_markdown))

# Audit Q/A Report
> Ahold Delhaize Annual Report 2024
## Could you specify the time frame these financial statements represent?
The financial statements represent the financial year ending on December 29, 2024, which consisted of 52 weeks.
> **Page 42**
> Net sales for the financial year ending on December 29, 2024, were €89.4bn, an increase of 0.7% compared to net sales for the year ending on December 31, 2023.

> **Page 42**
> Net sales for the financial year ending on December 29, 2024, were €89,356 million, an increase of €622 million, or 0.7%, compared to net sales of €88,734 million for the financial year ending on December 31, 2023.

> **Page 86**
> Ahold Delhaize's financial year is a 52- or 53-week period ending on the Sunday nearest to December 31 for the Company and our European operations, or the Saturday before the Sunday nearest to December 31 for our operations in the United States. The financial year 2024 consisted of 52 weeks and ended on December 29, 2024.

## What accounting standards or principles were applied in the preparation of these statements?
The financial statements have been prepared in accordance with International Financial Reporting Standards (IFRSs) as adopted by the EU and also comply with the financial reporting requirements included in Part 9 of Book 2 of the Dutch Civil Code.
> **Page 86**
> These financial statements have been prepared in accordance with International Financial Reporting Standards (IFRSs) as adopted by the EU and also comply with the financial reporting requirements included in Part 9 of Book 2 of the Dutch Civil Code.

## Have there been any alterations to accounting policies or practices since prior reporting periods?
Net sales, online sales and net sales growth for 2023 have been updated to reflect the change in presentation for media and data income. For details see Note 3. Financial information prior to 2023 has not been restated.
> **Page 41**
> Short-term borrowings and current portion of long-term debt comprise €1,444 million lease liabilities, €295 million short-term borrowings, €1,962 million bank overdrafts and €630 million current portion loans (for more information, see Note 26 to the consolidated financial statements).

## What process is used to determine and reveal major accounting estimates?
The financial reporting requires us to make judgements and use estimates and assumptions. All of the key estimates related to the financial statements and details of related sources can be found in the notes related to them in this report.
> **Page 267**
> Estimates and uncertainties
As we have elected to voluntarily comply with the CSRD, we are aware that the preparation of our sustainability statements is subject to Company-specific and temporary interpretations associated with this early compliance. While we prepared the sustainability statements based on the first set of ESRS, we recognize that a better understanding of the requirements may evolve when additional implementation guidance or Q&As are made available by the European Financial Reporting Advisory Group (EFRAG), and when market practice develops.

## What critical assumptions and judgments were utilized in the preparation of these financial statements?
Estimates and uncertainty include significant assumptions about the judgements used to determine metrics and data as well as material risks. More can be read on page 86.
> **Page 86**
> As we have elected to voluntarily comply with the CSRD, we are aware that the preparation of our sustainability statements is subject to Company-specific and temporary interpretations associated with this early compliance. While we prepared the sustainability statements based on the first set of ESRS, we recognize that a better understanding of the requirements may evolve when additional implementation guidance or Q&As are made available by the European Financial Reporting Advisory Group (EFRAG), and when market practice develops.

## How would you describe the organization's financial standing and performance as reflected in these statements?
In 2024, Ahold Delhaize achieved strong underlying results, showing positive net sales growth and all key financial goals for the year were met.
> **Page 9**
> We're thankful that, due to all of the initiatives I've mentioned so far, we could count on the loyalty of our brands' customers, and we met all of our key financial goals for the year.

> **Page 9**
> We delivered on our promises with a 4.0% underlying operating margin and a stable underlying EPS and overperformed on free cash flow at €2.5 billion.

## Do these financial results include any extraordinary or one-time items?
The (IFRS) operating income for 2024 include a list of the one-time adjustments.
> **Page 42**
> IFRS operating income was €2,784 million and our margin based on (IFRS) reporting operating income amounts to 3.1% of net sales, a decrease of 0.1 percentage points compared to last year. Operating margins were affected by one-time costs, primarily related to expenses from the closure of underperforming Stop & Shop stores, the transition of stores to affiliates under the Delhaize Belgium Future Plan and an amendment to, and additional funding for, the Dutch pension plan.

## Are there any transactions with related parties that are documented in the financial statements?
Ahold Delhaize has entered into arrangements with a number of its subsidiaries and affiliated companies. These arrangements relate to service transactions and financing agreements and were conducted at market prices.
> **Page 316**
> Ahold Delhaize has entered into arrangements with a number of its subsidiaries and affiliated companies in the course of its business. These arrangements relate to service transactions and financing agreements and were conducted at market prices.

## Could you detail the principal elements of operating expenses and their classification?
The principal elements of operating expenses and their classification is specified in the table as follows:
- Cost of product - 61,831 € million.
- Labor costs - 13,089 € million.
- Other operational expenses - 8,131 € million.
Depreciation and amortization - 3,477€ million.
> **Page 35**
> Operating expenses
In 2024, operating expenses increased by €31 million, or 0.1%, to €21,453 million, compared to €21,422 million in 2023. At constant exchange rates, operating expenses increased by 0.3%. As a percentage of net sales, operating expenses decreased by 0.1 percentage points to 24.0%, compared to 24.1% in 2023. Excluding gasoline sales and at constant exchange rates, operating expenses as a percentage of net sales decreased by 0.2 percentage points. The decrease in operating expenses as a percentage of sales in 2024, compared to 2023, was driven by sales leverage, strict cost control in all the brands and the conversion from integrated to affiliated stores as part of the Belgium Future Plan, partly offset by increasing labor inflation.

> **Page 42**
> Cost of sales
(65,551)
(64,880)
(671)
1.0%
Other income
431
414
17
4.2%
Operating expenses
(21,453)
(21,422)
(31)
0.1%
Operating income
2,784
2,846
(62)
(2.2)%

## Are there any asset impairments or write-downs shown in these statements?
Yes, net impairment losses of property, plant and equipment totalled  €229 million, is down by €147 million compared to 2023.
> **Page 46**
> Ahold Delhaize recorded the following impairments and reversals of impairments of assets - net in 2024 and 2023:  Impairment charges in 2024 were €229 million, down by €147 million compared to 2023.  These impairments mainly related to Stop & Shop stores in the U.S. and intangible assets and impairments related to bol and software in progress in Delhaize in Europe.

## What accounting treatment is applied to changes in fair value, particularly for financial instruments?
For financial instruments designated to be at fair value through profit or loss.A gain or loss on a financial asset measured at FVPL is recognized in the income statement for the reporting period in which it arises.
> **Page 314**
> A financial asset measured at FVPL is recognized initially at fair value and its transaction cost is recognized in profit or loss when incurred. A gain or loss on a financial asset measured at FVPL is recognized in the income statement for the reporting period in which it arises.

## Do the financial statements disclose any contingent liabilities or ongoing legal issues?
Ahold Delhaize and certain of its former or current subsidiaries are involved in a number of legal proceedings. These legal proceedings, whether pending, threatened or unasserted, if decided adversely or settled, may result in liability material to Ahold Delhaize's financial condition, results of operations or cash flows.
> **Page 325**
> Ahold Delhaize and certain of its former or current subsidiaries are involved in a number of legal proceedings, which include litigation as a result of divestments, tax and employment, as well as other litigation and inquiries. The legal proceedings discussed below, whether pending, threatened or unasserted, if decided adversely or settled, may result in liability material to Ahold Delhaize's financial condition, results of operations or cash flows.

## Are there any post-reporting period events that might affect the financial results?
The acquisition of Profi was completed on January 3, 2025, meaning the information will be recorded in the 2025 report.
> **Page 15**
> Founded in 2000, Profi Rom Food SRL (Profi) has grown to be a key player in Romania's retail sector. Profi supports almost 28,700 associates, working in around 1,700 stores across the country, including more than 1,500 partner-operated locations. Profi's mission is rooted in accessibility and convenience, with a focus on serving both urban and rural communities through a diverse portfolio of store formats, including Super, City, Go and Loco, which cater to the varied shopping needs of customers.

> **Page 15**
> Launched in early 2024, AD/01 is Ahold Delhaize's newest operational brand, developed to drive innovation and enhance the customer experience across Europe. With a focus on cutting-edge technologies, this tech studio plays a pivotal role in strengthening our capabilities as part of Ahold Delhaize's digital, data and tech ecosystem.

> **Page 42**
> When necessary, our brands take action to strengthen future performance, as they successfully did through the recent Belgium Future Plan and are working on through Stop & Shop's revitalization.

## How do these statements depict the organization's liquidity and cash flow situation?
The information regarding the group's cash flow situation is included in Free cash flow with stable underlying EPS and a €2.5 billion performance.
> **Page 9**
> We delivered on our promises with a 4.0% underlying operating margin and a stable underlying EPS and overperformed on free cash flow at €2.5 billion.

## Have there been any notable shifts in working capital ratios relative to previous periods?
Yes, Ahold Delhaize reported on decreased working capital due to an incidental receivable related to a tax refund in Belgium last year. However, as a % of net sales the change was only -0.2% (4.9% - 4.7%).
> **Page 8**
> The contribution by segment was 63% by the U.S., and 37% by Europe. The decrease in underlying operating income as well due to labor, as a result of wage level increases.

## Could you describe the organization's debt composition and the presentation of interest expenses?
Ahold Delhaize's net debt was €14,129 million as of December 29, 2024 - a decrease of €137 million from December 31, 2023.  The decrease in net debt was mainly the result of the strong free cash flow generation.
> **Page 40**
> In 2024, gross debt increased by €2,549 million to €20,315 million, primarily due to the issuance of €1,600 million new bonds and the increase of our short-term borrowings related to the cash held under a notional cash pooling arrangement, partly offset by the repayment of a €750 million bond at maturity.

> **Page 40**
> Ahold Delhaize's net debt was €14,129 million as of December 29, 2024 - a decrease of €137 million from December 31, 2023. The decrease in net debt was mainly the result of the strong free cash flow generation.

## What accounting and presentation methods are used for taxes in these financial statements?
The best estimate of the expenditure required to settle the obligation is reported within these financial statements. More can be read on page 332.
> **Page 332**
> The following table specifies the current and deferred tax components of income taxes in the income statement:  Total income taxes (expense) benefit 

> **Page 332**
> Provisions are recognized when (i) the Company has a present (legal or constructive) obligation as a result of past events, (ii) it is probable that an outflow of resources will be required to settle the obligation, and (iii) the amount can be reliably estimated.

> **Page 332**
> The amount recognized is the best estimate of the expenditure required to settle the obligation. Provisions are discounted whenever the effect of the time value of money is significant.

## Are there any contractual commitments or obligations outlined in these statements?
Ahold Delhaize has significant volume commitments. To read more about the details, review page 323.
> **Page 323**
> Ahold Delhaize enters into purchase commitments with vendors in the ordinary course of business. The Company has purchase contracts with some vendors for varying terms that require Ahold Delhaize to buy services and predetermined volumes of goods and goods not-for-resale at fixed prices.

> **Page 323**
> As of December 29, 2024, the Company's purchase commitments were approximately €3.8 billion (December 31, 2023: €3.6 billion).

## Could you provide a detailed breakdown of equity, including movements in retained earnings?
You can find it on page 291. It shows items for 2023 and 2024.
> **Page 291**
> A report on the changes, please see the attached chart in page 291

## If relevant, what accounting treatment is applied to foreign currency transactions and translation?
The assets and liabilities of subsidiaries with a functional currency other than the euro are translated into euros using the exchange rates prevailing at the balance sheet date.
> **Page 86**
> The financial statements of subsidiaries, joint ventures and associates are prepared in their functional currencies, which are determined based on the primary economic environment in which they operate. Transactions in currencies other than the functional currency are recorded at the rates of exchange prevailing on the transaction dates.

> **Page 86**
> At each balance sheet date, monetary items denominated in foreign currencies are translated into the entity's functional currency at the then prevailing rates.

## Are there any significant differences or contradictions found within these financial statements?
I am sorry, I cannot describe any significant differences or contradictions found within these financial statements.
## Could you explain the organization's approach to recognizing revenue and its implementation?
Revenue is recognized once the performance obligation happens. To what and how this happens, read page 272.
> **Page 272**
> Ahold Delhaize generates and recognizes net sales to retail customers as it satisfies its performance obligation at the point of sale in its stores and upon delivery of goods through its online channel. The Company also generates revenues from the sale of products to retail franchisees and affiliates that are recognized upon delivery. Ahold Delhaize recognizes fees from franchisees and affiliates as revenue as services are performed or the granted rights are used.

## What process is used to determine and disclose provisions for doubtful accounts and bad debts?
Ahold Delhaize uses a variety of factors in order to measure the proper level of collectibility. Read note 18 for details.
> **Page 289**
> The receivable balances are presented net of accounts payable and subject to an enforceable netting arrangement between the Company and the counterparty. The total effect of netting as of December 29, 2024, is €315 million (December 31, 2023: €319 million).

## Are there any major events or transactions that might affect the ability to compare these statements with others?
Ahold Delhaize is under a lot of restructuring and reorganization to improve cost.
> **Page 9**
> To make it possible for us to invest in the customer experience and drive innovation, we continued to simplify our organization in 2024. We've begun to see results from transformational projects, such as the Belgium Future Plan, and are executing on a plan to ensure a stable and thriving future for Stop & Shop.

## How does the organization handle risk management, and how is this communicated?
Ahold Delhaize integrates risk management into it's business. To know and learn more about how the processes align, review page 226.
> **Page 226**
> Through our ERM program, which is embedded in the execution of our strategy, the leadership of each of the brands and global functions review their principal risks

## What accounting treatment is used for gains or losses from asset disposals in these statements?
Ahold Delhaize recorded the gains and losses from the sale of assets - net in 2024 and 2023 and the amount recorded and what it was attributable to.
> **Page 46**
> Ahold Delhaize recorded the following (gains) losses on leases and the sale of assets - net in 2024 and 2023:

## Are there any non-operating revenues or expenses that influence these financial statements?
Net financial expenses in 2024 were up by €16 million compared to 2023.  The increase was primarily due to higher interest expense on the additional debt related to the acquisition of Profi and on leases, partially offset by higher interest income from higher cash levels.
> **Page 47**
> Net financial expenses Net financial expenses in 2024 were up by €16 million, or 2.9%, to €562 million, compared to €546 million in 2023. The increase was primarily due to higher interest expense on the additional debt related to the acquisition of Profi and on leases, partially offset by higher interest income from higher cash levels.

## Could you describe the nature and extent of the audit or review procedures carried out on these statements?
We have exercised professional judgment and have maintained professional skepticism throughout the audit, in accordance with Dutch Standards on Auditing, ethical requirements and independence requirements.
> **Page 346**
> We have exercised professional judgement and have maintained professional skepticism throughout the audit, in accordance with Dutch Standards on Auditing, ethical requirements and independence requirements.



In [7]:
df = pd.DataFrame(
    [
        answer.model_dump() for answer in audit_answers.answers
    ]
)
df

,question,answer,sources
0,Could you specify the time frame these financi...,The financial statements represent the financi...,"[{'page': 42, 'citation': 'Net sales for the f..."
1,What accounting standards or principles were a...,The financial statements have been prepared in...,"[{'page': 86, 'citation': 'These financial sta..."
2,Have there been any alterations to accounting ...,"Net sales, online sales and net sales growth f...","[{'page': 41, 'citation': 'Short-term borrowin..."
3,What process is used to determine and reveal m...,The financial reporting requires us to make ju...,"[{'page': 267, 'citation': 'Estimates and unce..."
4,What critical assumptions and judgments were u...,Estimates and uncertainty include significant ...,"[{'page': 86, 'citation': 'As we have elected ..."
5,How would you describe the organization's fina...,"In 2024, Ahold Delhaize achieved strong underl...","[{'page': 9, 'citation': 'We're thankful that,..."
6,Do these financial results include any extraor...,The (IFRS) operating income for 2024 include a...,"[{'page': 42, 'citation': 'IFRS operating inco..."
7,Are there any transactions with related partie...,Ahold Delhaize has entered into arrangements w...,"[{'page': 316, 'citation': 'Ahold Delhaize has..."
8,Could you detail the principal elements of ope...,The principal elements of operating expenses a...,"[{'page': 35, 'citation': 'Operating expenses ..."
9,Are there any asset impairments or write-downs...,"Yes, net impairment losses of property, plant ...","[{'page': 46, 'citation': 'Ahold Delhaize reco..."


In [11]:
display(
    Markdown(
        pd.DataFrame(
            [
                {
                    "answer": "this is the answer! <a href='https://www.aholddelhaize.com/media/wcqil04n/ad_annual-report_2024_interactive.pdf#page=30'>link</a>",
                }
            ]
        ).to_markdown()
    )
)

|    | answer                                                                                                                            |
|---:|:----------------------------------------------------------------------------------------------------------------------------------|
|  0 | this is the answer! <a href='https://www.aholddelhaize.com/media/wcqil04n/ad_annual-report_2024_interactive.pdf#page=30'>link</a> |

In [12]:
display(
    Markdown(df.to_markdown())
)

|    | question                                                                                                            | answer                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | sources                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|---:|:--------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | - Could you specify the time frame these financial statements represent?                                            | The financial statements represent the period from January 1 to December 29, 2024.                                                                                                                                                                                                                                                                                                                                                                                                      | [{'page': 3, 'citation': 'Ahold Delhaize Annual Report 2024'}, {'page': 5, 'citation': 'Ahold Delhaize Annual Report 2024'}]                                                                                                                                                                                                                                                                                                                       |
|  1 | - What accounting standards or principles were applied in the preparation of these statements?                      | The financial statements have been prepared in accordance with International Financial Reporting Standards (IFRSs) as adopted by the EU and also comply with the financial reporting requirements included in Part 9 of Book 2 of the Dutch Civil Code.                                                                                                                                                                                                                                 | [{'page': 264, 'citation': 'These financial statements have been prepared in accordance with International Financial Reporting Standards (IFRSs) as adopted by the EU and also comply with the financial reporting requirements included in Part 9 of Book 2 of the Dutch Civil Code.'}]                                                                                                                                                           |
|  2 | - Have there been any alterations to accounting policies or practices since prior reporting periods?                | Here are the alternations to accounting policies or practices since prior reporting periods: As of 2024, media and data income that was previously presented as part of Other income is presented as part of Net sales. The amendments do not have a significant impact on the Company's consolidated financial statements.                                                                                                                                                             | [{'page': 42, 'citation': 'Comparative amounts have been restated (see Note 3 to the consolidated financial statements).'}, {'page': 43, 'citation': "Our European brands' net sales consist of sales to consumers and to franchise stores."}, {'page': 327, 'citation': 'There are no other IFRSs that have been issued but are not yet effective that are expected to have a material effect on the future consolidated financial statements.'}] |
|  3 | - What process is used to determine and reveal major accounting estimates?                                          | Here is the process used to determine and reveal major accounting estimates: The preparation of financial statements requires management to make a number of estimates and assumptions that affect the reported amounts of assets and liabilities, revenues and expenses, and the disclosure of contingent assets and liabilities which will seldom equal the actual results.                                                                                                           | [{'page': 265, 'citation': 'The preparation of financial statements requires management to make a number of estimates and assumptions that affect the reported amounts of assets and liabilities, revenues and expenses, and the disclosure of contingent assets and liabilities which will seldom equal the actual results.'}]                                                                                                                    |
|  4 | - What critical assumptions and judgments were utilized in the preparation of these financial statements?           | Here are the critical assumptions and judgments utilized in the preparation of these financial statements: All assumptions, expectations and forecasts used as a basis for certain estimates within these financial statements represent good faith assessments of Ahold Delhaize’s current and future performance for which management believes there is a reasonable basis.                                                                                                           | [{'page': 265, 'citation': 'All assumptions, expectations and forecasts used as a basis for certain estimates within these financial statements represent good faith assessments of Ahold Delhaize’s current and future performance for which management believes there is a reasonable basis.'}]                                                                                                                                                  |
|  5 | - How would you describe the organization's financial standing and performance as reflected in these statements?    | Ahold Delhaize achieved strong underlying results, showing positive net sales growth (0.9% at constant rates) while sustaining a solid underlying operating margin, customers meet all of their key financial goals for the year. We delivered on our promises with a 4.0% underlying operating margin and a stable underlying EPS and overperformed on free cash flow at €2.5 billion.                                                                                                 | [{'page': 9, 'citation': 'We delivered on our promises with a 4.0% underlying operating margin and a stable underlying EPS and overperformed on free cash flow at €2.5 billion.'}]                                                                                                                                                                                                                                                                 |
|  6 | - Do these financial results include any extraordinary or one-time items?                                           | The financial statements do not include extraordinary or one-time items.                                                                                                                                                                                                                                                                                                                                                                                                                | [{'page': 42, 'citation': 'Underlying operating income and our margin based on (IFRS) reporting operating income amounts to 3.1% of net sales, a decrease of 0.1 percentage points compared to last year. Operating margins were affected by one-time costs'}]                                                                                                                                                                                     |
|  7 | - Are there any transactions with related parties that are documented in the financial statements?                  | Yes, there are related party transactions, with significant detail being presented in Note 31.                                                                                                                                                                                                                                                                                                                                                                                          | [{'page': 316, 'citation': 'Ahold Delhaize has entered into arrangements with a number of its subsidiaries and affiliated companies in the course of its business. These arrangements relate to service transactions and financing agreements and were conducted at market prices.'}]                                                                                                                                                              |
|  8 | - Could you detail the principal elements of operating expenses and their classification?                           | The principal elements of operating expenses are selling and general expenses, as follows; Selling expenses (i.e. wages) was 17,192 and General and administrative expenses was 4,261 million euros.                                                                                                                                                                                                                                                                                    | [{'page': 259, 'citation': 'Selling expenses 2 (17,192) (17,320) General and administrative expenses 2 (4,261) (4,102)'}]                                                                                                                                                                                                                                                                                                                          |
|  9 | - Are there any asset impairments or write-downs shown in these statements?                                         | Ahold Delhaize recorded the following impairment losses and reversals of impairments of assets: The United States €(38), Europe €219, Ahold Delhaize Group (181)                                                                                                                                                                                                                                                                                                                        | [{'page': 43, 'citation': 'The losses on leases and the sale of assets in 2024 were €181 million.'}]                                                                                                                                                                                                                                                                                                                                               |
| 10 | - What accounting treatment is applied to changes in fair value, particularly for financial instruments?            | The accounting treatment applied to changes in fair value, particularly for financial instruments: All derivative financial instruments are initially recognized at fair value at the date the derivative contracts are entered into and are subsequently remeasured to their fair value at the end of each reporting period. Gains and losses resulting from the fair value remeasurement are recognized in the income statement as fair value gains (losses) on financial instruments | [{'page': 309, 'citation': 'All derivative financial instruments are initially recognized at fair value at the date the derivative contracts are entered into and are subsequently remeasured to their fair value at the end of each reporting period. Gains and losses resulting from the fair value remeasurement are recognized in the income statement as fair value gains (losses) on financial instruments'}]                                |
| 11 | - Do the financial statements disclose any contingent liabilities or ongoing legal issues?                          | There are contingent liabilities, and they are specified in the commitments and contingencies section note 34.                                                                                                                                                                                                                                                                                                                                                                          | [{'page': 323, 'citation': 'Ahold Delhaize and its subsidiaries may be contingently liable for leases that have been assigned and/or transferred to third parties in connection with facility closings and dispositions. Ahold Delhaize could be required to perform the financial obligations under these leases if any of the third parties are unable to fulfill their lease obligations.'}]                                                    |
| 12 | - Are there any post-reporting period events that might affect the financial results?                               | The Ahold Delhaize report states acquisition is completed on January 3, 2025. Therefore this is the post-reporting event that might affect the financial results.                                                                                                                                                                                                                                                                                                                       | [{'page': 7, 'citation': 'We also acquired Romanian omnichannel brand Profi, which joined our family of brands on January 3, 2025. Together with Mega Image, they serve both rural and urban areas of this steadily growing market.'}, {'page': 46, 'citation': 'In 2024, as well as making strategic initiatives, with store openings and with acquisitions like Profi bringing scale and synergies.'}]                                           |
| 13 | - How do these statements depict the organization's liquidity and cash flow situation?                              | Ahold Delhaize states that operating cash flows from continuing operations amounted to €6,224 million. This indicates the amount of cash generated from its main business activities. Free cash flow is reported to be €2,545 million, showing the cash flow available for discretionary uses. Also the entity had strong net income.                                                                                                                                                   | [{'page': 41, 'citation': 'The most important component to note would be: Free cash flow, After €2.3 billion cash capital expenditure €2.5bn'}, {'page': 306, 'citation': 'Cash and cash equivalents include all cash-on-hand balances, checks, debit and credit card receivables, short-term highly liquid cash investments, and time deposits with original maturities of three months or less.'}]                                               |
| 14 | - Have there been any notable shifts in working capital ratios relative to previous periods?                        | I do not have the information necessary to answer your question. There are not notable shifts in working capital ratios relative to previous periods.                                                                                                                                                                                                                                                                                                                                   | []                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 15 | - Could you describe the organization's debt composition and the presentation of interest expenses?                 | Here is the organization's debt composition and the presentation of interest expenses, The total amount of loans (gross debt) reported as the end of 2024 is €20,315 million (a decrease from 2023 value).                                                                                                                                                                                                                                                                              | [{'page': 41, 'citation': 'Loans and Net Debt'}]                                                                                                                                                                                                                                                                                                                                                                                                   |
| 16 | - What accounting and presentation methods are used for taxes in these financial statements?                        | Ahold Delhaize's financial statements for 2024 reflect the income tax expense on continuing operations €10 (481), with an Effective income tax rate on continuing operations 21.7%.                                                                                                                                                                                                                                                                                                     | [{'page': 38, 'citation': 'In both the U.S. and Europe, declining inflation started at the end of 2022 and continued in 2024.'}, {'page': 46, 'citation': 'In 2024, income tax expense was €481 million, €25 million higher compared to €456 million in 2023.'}, {'page': 380, 'citation': 'There are more comments on that in the Sustainability section.'}]                                                                                      |
| 17 | - Are there any contractual commitments or obligations outlined in these statements?                                | Yes, there is a mention of contractual commitments in the PDF report.                                                                                                                                                                                                                                                                                                                                                                                                                   | [{'page': 323, 'citation': 'Ahold Delhaize has several significant contractual and financial commitments. Examples are the $0.5 billion commitment regarding supply agreements in the U.S. for renewable energy from new build resources for long terms (greater than 10 years.'}]                                                                                                                                                                 |
| 18 | - Could you provide a detailed breakdown of equity, including movements in retained earnings?                       | Here is a detailed breakdown of equity, including movements in retained earnings: Balance as of December 31, 2023 € 14,755 million. Net income attributable to common shareholders 1,874 Total other comprehensive income (loss) attributable to common shareholders (539)                                                                                                                                                                                                              | [{'page': 261, 'citation': 'Total equity and liabilities'}]                                                                                                                                                                                                                                                                                                                                                                                        |
| 19 | - If relevant, what accounting treatment is applied to foreign currency transactions and translation?               | For the foreign currency translation, relevant accounting treatment is The assets and liabilities of subsidiaries with a functional currency other than the euro are translated into euros using the exchange rates prevailing at the balance sheet date. Income and expense items are translated at the average exchange rates for the respective periods.                                                                                                                             | [{'page': 267, 'citation': 'Foreign currency translation The financial statements of subsidiaries, joint ventures and associates are prepared in their functional currencies, which are determined based on the primary economic environment in which they operate.'}]                                                                                                                                                                             |
| 20 | - Are there any significant differences or contradictions found within these financial statements?                  | I am sorry, but I cannot find any significant differences or contradictions within these financial statements.                                                                                                                                                                                                                                                                                                                                                                          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 21 | - Could you explain the organization's approach to recognizing revenue and its implementation?                      | Here's the organization's approach to recognizing revenue and its implementation Ahold Delhaize generates and recognizes net sales to retail customers as it satisfies its performance obligation at the point of sale in its stores and upon delivery of goods through its online channel.                                                                                                                                                                                             | [{'page': 267, 'citation': 'Ahold Delhaize generates and recognizes net sales to retail customers as it satisfies its performance obligation at the point of sale in its stores and upon delivery of goods through its online channel.'}]                                                                                                                                                                                                          |
| 22 | - What process is used to determine and disclose provisions for doubtful accounts and bad debts?                    | All receivables are stated net of allowances for potential losses on collection. To determine the amount of such allowances, historical collection experience is used, and it is recognized when it is probable that all or part of an account will not be collected                                                                                                                                                                                                                    | [{'page': 289, 'citation': 'The receivable balances are presented net of accounts payable and subject to an enforceable netting arrangement between the Company and the counterparty.'}, {'page': 310, 'citation': 'Ahold Delhaize has no significant concentrations of credit risk.'}]                                                                                                                                                            |
| 23 | - Are there any major events or transactions that might affect the ability to compare these statements with others? | Here are major events or transactions that might affect the ability to compare these statements with others: Note 3 has the comparative amounts have been restated to conform to the current year's presentation. (Net sales, online sales and net sales growth in local currency.                                                                                                                                                                                                      | [{'page': 41, 'citation': 'Group key financial indicators'}]                                                                                                                                                                                                                                                                                                                                                                                       |
| 24 | - How does the organization handle risk management, and how is this communicated?                                   | The organization handles risk management through a Enterprise Risk Management Program. Details on the objectives, identification, assessment and mitigating actions related to risk are in the PDF.                                                                                                                                                                                                                                                                                     | [{'page': 71, 'citation': 'risks and opportunities'}, {'page': 73, 'citation': 'Our Enterprise Risk Management (ERM) program has been created in line with our values and ethical principles.'}]                                                                                                                                                                                                                                                   |
| 25 | - What accounting treatment is used for gains or losses from asset disposals in these statements?                   | Gains and losses on the sale of assets - net are included in Net Financial Expenses. For further details, see Note 8                                                                                                                                                                                                                                                                                                                                                                    | [{'page': 43, 'citation': 'Ahold Delhaize recorded the following (gains) losses on leases and the sale of assets - net in 2024 and 2023:'}]                                                                                                                                                                                                                                                                                                        |
| 26 | - Are there any non-operating revenues or expenses that influence these financial statements?                       | Yes, there are non-operating revenues or expenses that influence the financial statements.                                                                                                                                                                                                                                                                                                                                                                                              | [{'page': 47, 'citation': 'Net financial expenses'}]                                                                                                                                                                                                                                                                                                                                                                                               |
| 27 | - Could you describe the nature and extent of the audit or review procedures carried out on these statements?       | We conducted our audit in accordance with Dutch law, including the Dutch Standards on Auditing. Our responsibilities under those standards are further described in the 'Our responsibilities for the audit of the financial statements' section of our report.                                                                                                                                                                                                                         | [{'page': 339, 'citation': "We conducted our audit in accordance with Dutch law, including the Dutch Standards on Auditing. Our responsibilities under those standards are further described in the 'Our responsibilities for the audit of the financial statements' section of our report."}]                                                                                                                                                     |

In [13]:
df.to_csv("./ad_qa_report.csv", index=False)